# MACHINE LEARNING OLS

In [17]:
#Importing libaries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from scipy import stats
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

In [3]:
#Importing data
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_column', None)
pd.set_option('display.max_colwidth', None)
csv_file_path = '../Modeldata/data_ready.csv'
df_final = pd.read_csv(csv_file_path)
df_final
df_final.set_index('Unnamed: 0', inplace=True)
df_final.index.name = None
print(len(df_final))

19848


In [5]:
##Deleting observations with less than 20 m2 -> To avoid "non-appartments"
outliers_m2 = df_final[(df_final['m2'] < 20) | ((df_final['m2'] < 50) & (df_final['Boligtype_Landejendom'] == 1))]
#print(outliers_m2)
df_final = df_final.drop(outliers_m2.index)

#Square meter price:
df_final['price/m2'] = df_final['Price']/df_final['m2']

print(len(df_final))
########################################################################
# Calculate Z-scores for each value in the column
df_final['z_score_price_m2'] = stats.zscore(df_final['price/m2'])

# Define outliers as any points with a Z-score > 4 or < -4
outliers = df_final[(df_final['z_score_price_m2'] > 4) | (df_final['z_score_price_m2'] < -4)]
#print(outliers.sort_values(by='price/m2', ascending=False))

df_final = df_final.drop(outliers.index)
df_final.drop(columns='price/m2', inplace=True)
########################################################################
# Calculate Z-scores for m2
df_final['z_score_m2'] = stats.zscore(df_final['m2'])

# Define outliers as any points with a Z-score > 4 or < -4
outliers_m2 = df_final[(df_final['z_score_m2'] > 4) | (df_final['z_score_m2'] < -4)]
#print(outliers_m2.sort_values(by='m2', ascending=True))
print(len(outliers_m2))
df_final = df_final.drop(outliers_m2.index)
print(len(df_final))

########################################################################
"""
#Calculate Z-scores for price
df_final['z_score_price'] = stats.zscore(df_final['Price'])
# Define outliers as any points with a Z-score > 4 or < -4
outliers_price = df_final[(df_final['z_score_price'] > 4) | (df_final['z_score_price'] < -4)]
#print(outliers_m2.sort_values(by='m2', ascending=True))
df_final = df_final.drop(outliers_price.index)

"""
print(len(df_final))

df_final = df_final.drop('z_score_m2', axis=1)
df_final = df_final.drop('z_score_price_m2', axis=1)

#print(df_final.sort_values(by='price/m2', ascending=True))

19845
97
19631
19631


In [7]:
y = df_final['Price']
X = df_final.drop(columns=['Price'])

In [9]:
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=1) #The random_state works as a seed

#2
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=0.5, random_state=1)

In [11]:
# LOAD DATA FROM SCIKIT-LEARN
from sklearn.linear_model import Lasso, LinearRegression

# DEFINE PIPELINE
pipe_lr = make_pipeline(PolynomialFeatures(), 
                        StandardScaler(),
                        LinearRegression())

# FIT PIPELINE TO DEVELOPMENT DATA
pipe_lr.fit(X_dev, y_dev)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [13]:
# LOAD DATA FROM SCIKIT-LEARN
from sklearn.metrics import mean_squared_error as mse

# PREPARATION
perform = [] # Store performance
lambdas = np.logspace(0, 7, 66) # Grid of lambdas

# FOR EACH LAMBDA, FIT A MODEL ON TRAINING DATA, CHECK PERFORMANCE ON VALIDATION AND STORE MSE
for lambda_ in lambdas:
    pipe_lasso = make_pipeline(PolynomialFeatures(), 
                               StandardScaler(),
                               Lasso(alpha=lambda_, random_state=161193))
    pipe_lasso.fit(X_train, y_train)
    y_pred = pipe_lasso.predict(X_val)
    perform.append(mse(y_pred, y_val))

# CRATE A SERIES WITH PERFORMANCE AND FIND PARM WITH LOWEST MSE    
hyperparam_perform = pd.Series(perform,index=lambdas)
optimal = hyperparam_perform.nsmallest(1)    
print('Optimal lambda:', optimal.index[0])
print('Validation MSE: %.3f' % optimal.values[0])

Optimal lambda: 54759.76030421944
Validation MSE: 1887333151645.778


In [15]:
# INSERT OPTIMAL LAMBDA INTO NEW MODEL
pipe_lasso = make_pipeline(PolynomialFeatures(), 
                           StandardScaler(),
                           Lasso(alpha=optimal.index[0]))

# FIT NEW MODEL ON ALL OF THE DEVELOPMENT (non-test) DATA TO BUILD BEST POSSIBLE MODEL
pipe_lasso.fit(X_dev, y_dev) 

# COMPARE MODEL PERFORMANCE ON TEST DATA
print('Lasso', round(mse(pipe_lasso.predict(X_test),y_test), 2))
print('LinReg', round(mse(pipe_lr.predict(X_test),y_test), 2))

Lasso 18925304222714.69
LinReg 8.858772648950795e+36


In [21]:
y_test_pred_lasso = pipe_lasso.predict(X_test)
r2_score_lasso = r2_score(y_test, y_test_pred_lasso)

In [19]:
RMSE_lasso = np.sqrt(round(mse(pipe_lasso.predict(X_test),y_test), 2))
print(RMSE_lasso)

4350322.3125091195


In [23]:
print(r2_score_lasso)

-1.386961336622158
